# Fundamentos de BD (Márcio Victorino)

## Partes do Projeto:
1. Introdução.
1. Modelo de dados Relacional.
1. O script SQL que gerou o banco de dados.
1. Apresentar o processo de ETL (Extract, Transform, Load) para importação dos
dados para o banco de dados.
1. Utilização de pelo menos uma View.
1. Utilização de pelo menos uma Procedure (com comandos condicionais).
1. Utilização de pelo menos um trigger (com comandos condicionais).
1. No mínimo 5 Consultas SQL (a complexidade da consulta será avaliada).

**Data Limite para a Entrega da Parte Escrita: 02/06/2023**

## Observações

- Dados abertos (ex: FNDE, bolsa família)
- Linkar com o Artigo da matéria do Ladeira MD
- Joga em um CSV e alimenta o BD SQL
- No mínimo milhares de dezenas de linhas
- Não executar a consulta na apresentação
- Printar os resultados e mostrar na apresentação

## Alunos: 

Marcelo Anselmo de Souza Filho
- Matrícula: **231109719**
- Email: **marcelofilho@mpf.mp.br**

Arivaldo Gonçalves
- Matrícula: **231109620**
- Email: **arivaldofreitas@correios.com.br**

Luciana Maria
- Matrícula: **231109700**
- Email: **luciana@mpdft.mp.br**

____

# 1. Introdução

Este estudo aborda a diferença salarial entre homens e mulheres na área de TI durante a pandemia. Ele também explora possíveis cenários para analisara disparidade salarial e de desligamento entre gêneros na área de tecnologia, antes e após a pandemia.	Utilizou-se dados no nível do indivíduo, de 2019 e 2020, obtidos da Relação Anual de Informações Sociais (Rais), que proporciona dados oficiais sobre o mercado de trabalho no Brasil. No geral, constatou-se a remuneração média das mulheres é menor que a dos homens em todas as regiões do Brasil, tanto em 2019 quanto em 2020. Além disso, a quantidade de desligamento de homens é maior que a de mulheres em todas as regiões do Brasil, tanto em 2019 quanto em 2020.

# 2. Modelo de dados Relacional

## Carregar dados

In [1]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

df = pd.read_parquet(
    "../output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip")

qnt_full = len(df)
print(f"""
Quantidade docs: {qnt_full}
""")

df.head(2)



Quantidade docs: 1543009



,ano,sigla_uf_id,regiao,regiao_id,sigla_uf,remuneracao_media,desligamento,idade,ocupacao_id,ocupacao,sexo_id,sexo
0,2019,11,Centro-Oeste,0,MS,997.99,0,17,317210,Analista de service desk,1,Masculino
1,2019,11,Centro-Oeste,0,MS,0.00,0,36,212420,Analista de suporte,1,Masculino


In [2]:
# df.info()

## Modelo Entidade Relacionamento

https://mermaid.js.org/syntax/entityRelationshipDiagram.html

```mermaid
---
title: Modelo Entidade Relacionamento
---
erDiagram
    RAIS {
        int id PK
        int id_ocupacao FK
        int id_sexo FK
        int id_uf FK
        int ano
        float remuneracao_media
        int desligamento
        int idade
    }
    OCUPACAO {
        int id PK
        string nome
    }
    SEXO {
        int id PK
        string nome
    }
    UF {
        int id PK
        int id_regiao FK
        string sigla_uf
    }
    REGIAO {
        int id PK
        string nome
    }
    RAIS ||--|| OCUPACAO : contem
    RAIS ||--|| SEXO : contem
    RAIS ||--|| UF : contem
    UF ||--|| REGIAO : faz_parte
    
```

# Criando DB

https://dev.mysql.com/doc/connector-python/en/connector-python-example-ddl.html

In [6]:
# !pip install mysql-connector-python

In [7]:
from __future__ import print_function

import mysql.connector
from mysql.connector import errorcode

cnx = mysql.connector.connect(user='test', password='test')
cursor = cnx.cursor()

DB_NAME = 'test'

TABLES = {
    "OCUPACAO": ("""
        CREATE TABLE OCUPACAO (
            id INT NOT NULL,
            nome VARCHAR(255) NOT NULL,
            PRIMARY KEY (id)
        ) ENGINE=InnoDB
    """),
    "SEXO": ("""
        CREATE TABLE SEXO (
            id INT NOT NULL,
            nome VARCHAR(255) NOT NULL,
            PRIMARY KEY (id)
        ) ENGINE=InnoDB
    """),
    "REGIAO": ("""
        CREATE TABLE REGIAO (
            id INT NOT NULL,
            nome VARCHAR(255) NOT NULL,
            PRIMARY KEY (id)
        ) ENGINE=InnoDB
    """),
    "UF": ("""
        CREATE TABLE UF (
            id INT NOT NULL,
            id_regiao INT NOT NULL,
            nome VARCHAR(255) NOT NULL,
            PRIMARY KEY (id),
            FOREIGN KEY (id_regiao) REFERENCES REGIAO(id)
        ) ENGINE=InnoDB
    """),
    "RAIS": ("""
        CREATE TABLE RAIS (
            id INT NOT NULL AUTO_INCREMENT,
            id_ocupacao INT NOT NULL,
            id_sexo INT NOT NULL,
            id_uf INT NOT NULL,
            ano INT NOT NULL,
            remuneracao_media FLOAT NOT NULL,
            desligamento INT NOT NULL,
            idade INT NOT NULL,
            PRIMARY KEY (id),
            FOREIGN KEY (id_ocupacao) REFERENCES OCUPACAO(id),
            FOREIGN KEY (id_sexo) REFERENCES SEXO(id),
            FOREIGN KEY (id_uf) REFERENCES UF(id)
        ) ENGINE=InnoDB
    """),
}



In [8]:
# def conectando_bd():
#   cnx = mysql.connector.connect(user='test', password='test')
#   return cnx.cursor()

# cursor = conectando_bd()


In [9]:
def create_database(cursor):
    try:
        cursor.execute(
            "DROP DATABASE IF EXISTS {}".format(DB_NAME))
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

create_database(cursor)
print("Database {} created successfully.".format(DB_NAME))
cnx.database = DB_NAME

Database test created successfully.


In [10]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


Creating table OCUPACAO: OK
Creating table SEXO: OK
Creating table REGIAO: OK
Creating table UF: OK
Creating table RAIS: OK


# Inserindo Dados

### Carregando dados do CSV

In [11]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

df = pd.read_parquet(
    "../output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip")

qnt_total = len(df)

print(f"""
Quantidade docs: {qnt_total}
""")

df.head(2)



Quantidade docs: 1543009



,ano,sigla_uf_id,regiao,regiao_id,sigla_uf,remuneracao_media,desligamento,idade,ocupacao_id,ocupacao,sexo_id,sexo
0,2019,11,Centro-Oeste,0,MS,997.99,0,17,317210,Analista de service desk,1,Masculino
1,2019,11,Centro-Oeste,0,MS,0.00,0,36,212420,Analista de suporte,1,Masculino


### Inserindo no BD

In [12]:
# cursor = cnx.cursor()
# cursor.execute('SET NAMES utf8;')
# cursor.execute('SET CHARACTER SET utf8;')
# cursor.execute('SET character_set_connection=utf8;')


In [13]:
def get_coluna_id_valor(coluna_id, coluna_valor):
    counts = df[[coluna_id, coluna_valor]].value_counts()
    return counts.index.tolist()
    
get_coluna_id_valor("sexo_id", "sexo")

[(1, 'Masculino'), (2, 'Feminino')]

In [14]:
def get_uf_and_regiao_id():
    counts = df[["sigla_uf_id", "regiao_id", "sigla_uf"]].value_counts()
    return counts.index.tolist()

get_uf_and_regiao_id()[:2]


[(25, 3, 'SP'), (18, 3, 'RJ')]

In [15]:
from __future__ import print_function
from datetime import date, datetime, timedelta
import mysql.connector

cnx = mysql.connector.connect(user='test', password='test', database='test')
cursor = cnx.cursor()

# === OCUPACAO ===

add_query = ("INSERT INTO OCUPACAO "
            "(id, nome) "
            "VALUES (%s, %s)")

[cursor.execute(add_query, (id, valor))
 for id, valor in get_coluna_id_valor("ocupacao_id", "ocupacao")]
 
# === SEXO ===

add_query = ("INSERT INTO SEXO "
            "(id, nome) "
            "VALUES (%s, %s)")

[cursor.execute(add_query, (id, valor))
 for id, valor in get_coluna_id_valor("sexo_id", "sexo")]

# === REGIAO ===

add_query = ("INSERT INTO REGIAO "
            "(id, nome) "
            "VALUES (%s, %s)")

[cursor.execute(add_query, (id, valor))
 for id, valor in get_coluna_id_valor("regiao_id", "regiao")]

# ============ COMMIT ============
cnx.commit()



In [16]:

# === UF ===

add_query = ("INSERT INTO UF "
             "(id, id_regiao, nome) "
             "VALUES (%s, %s, %s)")

[cursor.execute(add_query, (id, id_regiao, valor))
 for id, id_regiao, valor in get_uf_and_regiao_id()]

# ============ COMMIT ============
cnx.commit()


IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`test`.`UF`, CONSTRAINT `UF_ibfk_1` FOREIGN KEY (`id_regiao`) REFERENCES `REGIAO` (`id`))

In [ ]:
import mysql.connector
from mysql.connector import errorcode

cnx = mysql.connector.connect(user='test', password='test', database='test')
cursor = cnx.cursor()

# from each line in df create a sql insert statement
def create_insert_sql(row):
    print(f"""{row['ocupacao_id']},
                {row['sexo_id']},
                {row['sigla_uf_id']},
                {row['ano']},
                {row['remuneracao_media']},
                {row['desligamento']},
                {row['idade']}""")

    add = ("INSERT INTO RAIS "
                "(id_ocupacao, id_sexo, id_uf, ano, remuneracao_media, desligamento, idade) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s)")

    data = ({row['ocupacao_id']},
            {row['sexo_id']},
            {row['sigla_uf_id']},
            {row['ano']},
            {row['remuneracao_media']},
            {row['desligamento']},
            {row['idade']})

    # Insert new employee
    cursor.execute(add, data)

# df[:1].apply(create_insert_sql, axis=1)
# df[:3].apply(cursor.execute(create_insert_sql), axis=1)
df[:3].apply(create_insert_sql, axis=1)


317210,
                1,
                11,
                2019,
                997.99,
                0,
                17


MySQLInterfaceError: Python type set cannot be converted

In [ ]:
from __future__ import print_function
from datetime import date, datetime, timedelta
import mysql.connector

cnx = mysql.connector.connect(user='test', password='test', database='test')
cursor = cnx.cursor()


# cursor.execute("INSERT INTO RAIS ( id_ocupacao, id_sexo, id_uf, ano, remuneracao_media, desligamento, idade ) VALUES ( 317110, 1, 11, 2019, 1547, 0, 21 )")
# cursor.execute(("INSERT INTO SEXO (id, nome) VALUES (%s, %s)", (1, 'Feminino')))
# cursor.execute("INSERT INTO SEXO (nome) VALUES (%s)", ('Feminino', 'Masculino'))

add_employee = ("INSERT INTO SEXO "
                "(id, nome) "
                "VALUES (%s, %s)")
data_employee = (1, 'Feminino')

# Insert new employee
cursor.execute(add_employee, data_employee)

# Make sure data is committed to the database
cnx.commit()


In [ ]:
from __future__ import print_function
from datetime import date, datetime, timedelta
import mysql.connector

cnx = mysql.connector.connect(user='test', password='test', database='test')
cursor = cnx.cursor()

tomorrow = datetime.now().date() + timedelta(days=1)

add_employee = ("INSERT INTO employees "
                "(first_name, last_name, hire_date, gender, birth_date) "
                "VALUES (%s, %s, %s, %s, %s)")
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))

# Insert new employee
cursor.execute(add_employee, data_employee)
emp_no = cursor.lastrowid

# Insert salary information
data_salary = {
    'emp_no': emp_no,
    'salary': 50000,
    'from_date': tomorrow,
    'to_date': date(9999, 1, 1),
}
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

cursor.close()
cnx.close()


In [ ]:
cursor.close()
cnx.close()